In [1]:
import numba
from numba import jit
import time
import numpy as np

Original Functions:

In [2]:
def minibatch(data, batch_size):
    '''Define a function to compute the minibatches'''
    n = data.shape[0]
    p = data.shape[1]
    if n % batch_size != 0:
        n = (n // batch_size) * batch_size
    ind = np.arange(n)
    np.random.shuffle(ind)
    n_batches = n // batch_size
    data = data[ind].reshape(batch_size, p, n_batches)
    return(data, n_batches)

In [3]:
def sghmc(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    "Stochastic Gradient Hamiltonian Monte Carlo"
    by Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    ICML (2014).
    
    The inputs include:
    grad_U = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    The return is a np.array of positions of theta.'''

    # initialize parameters and check if the inputs work
    p = len(theta_0) # get dimension of theta_0
    n = dat.shape[0] # get the row number of dat
    theta_samps = np.zeros((p, niter*(n // batch_size))) # set up matrix of 0s to hold samples
    # compute beta_hat and Sigma as described on page 6 in the SGHMC paper
    beta_hat = 0.5 * V_hat @ eta
    Sigma = 2 * (alpha - beta_hat) @ eta
    # check if the Sigma is a positive definite matrix
    if np.all(np.linalg.eigvals(Sigma)) <= 0: 
        print("Error: (alpha - beta_hat) eta is not positive definite")
        return
    # Need batch size to be <= the amount of data
    if (batch_size > n): 
        print("Error: batch_size must be <= number of data points")
        return
    
    # loop through algorithm to get niter samples
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1) # initialize nu
    theta = theta_0 # initialize theta
    it = 0
    for i in range(niter):
        dat_resh, nbatches = minibatch(dat, batch_size)
        
        # Resample momentum every epoch
        nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
        
        for batch in range(nbatches):
            grad_U_batch = grad_U(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ grad_U_batch - alpha @ nu + \
                 np.random.multivariate_normal(np.zeros(p), Sigma).reshape(p, -1)
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

In [4]:
def sghmc_cleaned(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    Stochastic Gradient Hamiltonian Monte Carlo 
    ICML 2014.
    The inputs are:
    grad_U = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    See paper for more details
    The return is a np.array of positions of theta.'''

    ### Initialization and checks ###
    # get dimension of the thing you're sampling
    p = len(theta_0)
    # set up matrix of 0s to hold samples
    n = dat.shape[0]
    theta_samps = np.zeros((p, niter*(n // batch_size)))
    # fix beta_hat as described on pg. 6 of paper
    beta_hat = 0.5 * V_hat @ eta
    # We're sampling from a N(0, 2(alpha - beta_hat) @ eta)
    # so this must be a positive definite matrix
    Sigma = 2 * (alpha - beta_hat) @ eta
    Sig_chol = np.linalg.cholesky(Sigma)
    if np.all(np.linalg.eigvals(Sigma)) <= 0: 
        print("Error: (alpha - beta_hat) eta is not positive definite")
        return
    # Need batch size to be <= the amount of data
    if (batch_size > n): 
        print("Error: batch_size must be <= number of data points")
        return

    # initialize nu and theta 
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
    theta = theta_0

    # set up for Chol decomp for MV normal sampling of nu every epoch
    eta_chol = np.linalg.cholesky(eta)

    # loop through algorithm to get niter samples
    it = 0
    for i in range(niter):
        dat_resh, nbatches = minibatch(dat, batch_size)
        
        # Resample momentum every epoch
        nu = np.sqrt(eta_chol) @ np.random.normal(size=p).reshape(p,-1) # sample from MV normal
        
        for batch in range(nbatches):
            grad_U_batch = grad_U(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ grad_U_batch - alpha @ nu + \
                 Sig_chol @ np.random.normal(size=p).reshape(p,-1) # sample from multivariate normal
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

Optimization, try to use JIT first

In [5]:
@jit
def minibatch_numba(data, batch_size):
    '''Define a function to compute the minibatches'''
    n = data.shape[0]
    p = data.shape[1]
    if n % batch_size != 0:
        n = (n // batch_size) * batch_size
    ind = np.arange(n)
    np.random.shuffle(ind)
    n_batches = n // batch_size
    data = data[ind].reshape(batch_size, p, n_batches)
    return(data, n_batches)

In [6]:
@jit
def sghmc_numba(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    "Stochastic Gradient Hamiltonian Monte Carlo"
    by Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    ICML (2014).
    
    The inputs include:
    grad_U = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    The return is a np.array of positions of theta.'''

    # initialize parameters and check if the inputs work
    p = len(theta_0) # get dimension of theta_0
    n = dat.shape[0] # get the row number of dat
    theta_samps = np.zeros((p, niter*(n // batch_size))) # set up matrix of 0s to hold samples
    # compute beta_hat and Sigma as described on page 6 in the SGHMC paper
    beta_hat = 0.5 * V_hat @ eta
    Sigma = 2 * (alpha - beta_hat) @ eta
    # check if the Sigma is a positive definite matrix
    if np.all(np.linalg.eigvals(Sigma)) <= 0: 
        print("Error: (alpha - beta_hat) eta is not positive definite")
        return
    # Need batch size to be <= the amount of data
    if (batch_size > n): 
        print("Error: batch_size must be <= number of data points")
        return
    
    # loop through algorithm to get niter samples
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1) # initialize nu
    theta = theta_0 # initialize theta
    it = 0
    for i in range(niter):
        dat_resh, nbatches = minibatch(dat, batch_size)
        
        # Resample momentum every epoch
        nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
        
        for batch in range(nbatches):
            grad_U_batch = grad_U(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ grad_U_batch - alpha @ nu + \
                 np.random.multivariate_normal(np.zeros(p), Sigma).reshape(p, -1)
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

In [7]:
@jit
def sghmc_cleaned(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    Stochastic Gradient Hamiltonian Monte Carlo 
    ICML 2014.
    The inputs are:
    grad_U = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    See paper for more details
    The return is a np.array of positions of theta.'''

    ### Initialization and checks ###
    # get dimension of the thing you're sampling
    p = len(theta_0)
    # set up matrix of 0s to hold samples
    n = dat.shape[0]
    theta_samps = np.zeros((p, niter*(n // batch_size)))
    # fix beta_hat as described on pg. 6 of paper
    beta_hat = 0.5 * V_hat @ eta
    # We're sampling from a N(0, 2(alpha - beta_hat) @ eta)
    # so this must be a positive definite matrix
    Sigma = 2 * (alpha - beta_hat) @ eta
    Sig_chol = np.linalg.cholesky(Sigma)
    if np.all(np.linalg.eigvals(Sigma)) <= 0: 
        print("Error: (alpha - beta_hat) eta is not positive definite")
        return
    # Need batch size to be <= the amount of data
    if (batch_size > n): 
        print("Error: batch_size must be <= number of data points")
        return

    # initialize nu and theta 
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
    theta = theta_0

    # set up for Chol decomp for MV normal sampling of nu every epoch
    eta_chol = np.linalg.cholesky(eta)

    # loop through algorithm to get niter samples
    it = 0
    for i in range(niter):
        dat_resh, nbatches = minibatch(dat, batch_size)
        
        # Resample momentum every epoch
        nu = np.sqrt(eta_chol) @ np.random.normal(size=p).reshape(p,-1) # sample from MV normal
        
        for batch in range(nbatches):
            grad_U_batch = grad_U(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ grad_U_batch - alpha @ nu + \
                 Sig_chol @ np.random.normal(size=p).reshape(p,-1) # sample from multivariate normal
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

In [8]:
#comparsion



In [9]:
%%file project.cpp
<%
cfg['compiler_args'] = ['-std=c++11']
cfg['include_dirs'] = ['../../eigen3']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/eigen.h>
#include <stdexcept>
#include <algorithm> // std::random_shuffle
#include <random>

#include <Eigen/LU>
#include <Eigen/Dense>

namespace py = pybind11;
using std::default_random_engine;
using std::normal_distribution;
        
default_random_engine re{100};
normal_distribution<double> norm(0, 1);
auto rnorm = bind(norm, re);

Eigen::MatrixXd rnorm_vec(int n) {
    Eigen::MatrixXd res_vec = Eigen::MatrixXd::Zero(n, 1);
    for (int i=0; i<n; i++) {
        res_vec(i,0) = rnorm();
    }
    return res_vec;
}
     
Eigen::MatrixXd gradU_mixNormals(Eigen::MatrixXd theta, Eigen::MatrixXd x, int n, int batch_size) {
    // calculate gradient for mixture of normals example
    int p = theta.rows();
    Eigen::ArrayXd c_0 = theta(0,0) - x.array();
    Eigen::ArrayXd c_1 = theta(1,0) - x.array();
    Eigen::ArrayXd star = 0.5 * (-0.5 * c_0.pow(2)).exp() + 0.5 * (-0.5 * c_1.pow(2)).exp();
    Eigen::ArrayXd star_prime;
    Eigen::MatrixXd grad = Eigen::MatrixXd::Zero(p, 1);
    for (int i=0; i<p; i++) {
        star_prime = 0.5 * (-0.5 * (theta(i,0) - x.array()).pow(2)).exp() * (theta(i,0) - x.array());
        grad(i,0) = -theta(i,0)/10 - (n/batch_size)*(star_prime/star).sum();
    }
    return grad;
} 

Eigen::MatrixXd sghmc(std::string gradU_choice, Eigen::MatrixXd eta, int niter, Eigen::MatrixXd alpha, Eigen::MatrixXd theta_0, Eigen::MatrixXd V_hat, Eigen::MatrixXd dat, int batch_size){
    int p = theta_0.rows(); // dimension of the thing you're sampling
    int n = dat.rows();     // number of data observations
    int p_dat = dat.cols(); // 2nd dimension of data
    int nbatches = n / batch_size; // how many batches data will be broken into
    Eigen::MatrixXd dat_temp = dat;
    Eigen::MatrixXd dat_batch = Eigen::MatrixXd::Zero(batch_size, p_dat);
    Eigen::MatrixXd gradU_batch = Eigen::MatrixXd::Zero(p, 1);
    Eigen::MatrixXd theta_samps = Eigen::MatrixXd::Zero(p, niter*(n/batch_size));
    std::vector<int> ind;
    // fix beta_hat as described on pg. 6 of paper
    Eigen::MatrixXd beta_hat = 0.5 * V_hat * eta;
    // We're sampling from a N(0, 2(alpha - beta_hat) @ eta)
    // so this must be a positive definite matrix
    Eigen::MatrixXd Sigma = 2.0 * (alpha - beta_hat) * eta;
    Eigen::LLT<Eigen::MatrixXd> lltOfSig(Sigma); // compute the Cholesky decomposition of Sigma
    if(lltOfSig.info() == Eigen::NumericalIssue){ // check if we got error, and break out if so
        return theta_samps; // will just give back all-0s
    }
    Eigen::MatrixXd Sig_chol = lltOfSig.matrixL(); // get L in Chol decomp
    if(batch_size > n){ // Need batch size to be <= the amount of data
        return theta_samps; // will just give back all-0s
    }
    // initialize more things! (nu and theta, to be specific)
    Eigen::LLT<Eigen::MatrixXd> lltOfeta(eta); // compute the Cholesky decomposition of eta
    Eigen::MatrixXd eta_chol = lltOfeta.matrixL(); // get L in Chol decomp
    Eigen::MatrixXd nu = eta_chol * rnorm_vec(p); // initialize nu
    Eigen::MatrixXd theta = theta_0; // initialize theta 
    
    // loop through algorithm to get niter*batch_size samples
    int big_iter = 0;
    for (int it=0; it<niter; it++) {
        
        // shuffle rows of dat to get dat_temp 
        Eigen::VectorXi indices = Eigen::VectorXi::LinSpaced(dat.rows(), 0, dat.rows());
        std::random_shuffle(indices.data(), indices.data() + dat.rows());
        dat_temp = indices.asPermutation() * dat;
        
        // Resample momentum every epoch
        nu = eta_chol * rnorm_vec(p); // sample from MV normal
        
        // loop through the batches
        int count_lower = 0;
        int count_upper = batch_size;
        for (int b=0; b<nbatches; b++){
            int batch_ind = 0;
            for (int ind_temp=count_lower; ind_temp<count_upper; ind_temp++){
                dat_batch.row(batch_ind) = dat_temp.row(ind_temp);
                batch_ind += 1;
            }
            count_lower += batch_size; // add batch size to each iterator
            count_upper += batch_size;
            if (gradU_choice == "fig1"){
                gradU_batch = gradU_noisyFig1(theta);
            } else if (gradU_choice == "mixture_of_normals"){
                gradU_batch = gradU_mixNormals(theta, dat_batch, n, batch_size);
            } else {
                return theta_samps; // will just give back all-0s
            }
            nu = nu - eta * gradU_batch - alpha * nu + Sig_chol * rnorm_vec(p);
            theta = theta + nu;
            theta_samps.col(big_iter) = theta;
            big_iter += 1;
        }
    }

    return theta_samps;
}
    
PYBIND11_MODULE(project, m) {
    m.doc() = "auto-compiled c++ extension";
    m.def("sghmc", &sghmc);
}

Overwriting project.cpp


In [10]:
#comparsion

